## Import

In [1]:
from re import A
import pandas as pd
import random
import os
import numpy as np

#knn, randomforest, decisiontree, logisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score


SEED = 37

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


seed_everything(SEED)  # Seed 고정

## Data Load

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv
Saving test.csv to test.csv
Saving sample_submission.csv to sample_submission.csv


In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [50]:
# train.info()
# test.info()
# pd.set_option('display.max_rows', None)
# dd = train.notnull().sum()
# print(dd)

PRODUCT_ID      598
Y_Class         598
Y_Quality       598
TIMESTAMP       598
LINE            598
PRODUCT_CODE    598
X_1             349
X_2             349
X_3             349
X_4             349
X_5             349
X_6             349
X_7             349
X_8             349
X_9             349
X_10            349
X_11            349
X_12            349
X_13            349
X_14            349
X_15            349
X_16            349
X_17            349
X_18            349
X_19            349
X_20            349
X_21            349
X_22            349
X_23            349
X_24            349
X_25            349
X_26            349
X_27            349
X_28            349
X_29            349
X_30            349
X_31            349
X_32            349
X_33            349
X_34            349
X_35            349
X_36            349
X_37            349
X_38            349
X_39            349
X_40            349
X_41            349
X_42            349
X_43            349
X_44            349


In [29]:
x_train = train
y_train = train['Y_Class']

#test set 사용
test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])


# drop_x = []
# for i in range(1, 2876):
#   tmp = 'X_'+str(i)
#   if len(x_train[tmp].unique()) > 16: #기준을 더 낮추어봄
#     pass
#   else:
#     drop_x.append(tmp)

x_train = x_train.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
# x_train = x_train.drop(columns = drop_x)
# test_x = test_x.drop(columns = drop_x)

x_train = x_train.fillna(0)
test_x = test_x.fillna(0)

#split 분할 코드
# x_train, x_test, y_train, y_test = tts(x_train, y_train, test_size=0.2, random_state = 1)



## Data Pre-processing

In [30]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(x_train[i])
    x_train[i] = le.transform(x_train[i])
    
    # for label in np.unique(x_test[i]): 
    #     if label not in le.classes_: 
    #         le.classes_ = np.append(le.classes_, label)
    # x_test[i] = le.transform(x_test[i])
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])  
print('Done.')

Done.


In [31]:
print(x_train.columns)
print(test_x.columns)

Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7',
       'X_8',
       ...
       'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
       'X_2873', 'X_2874', 'X_2875'],
      dtype='object', length=2877)
Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7',
       'X_8',
       ...
       'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
       'X_2873', 'X_2874', 'X_2875'],
      dtype='object', length=2877)


## Feature Scaling

## Classification Model Fit

In [32]:
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state = 0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators = 100)


knn_clf.fit(x_train, y_train)
rf_clf.fit(x_train, y_train)
dt_clf.fit(x_train, y_train)
ada_clf.fit(x_train, y_train)


AdaBoostClassifier(n_estimators=100)

## CV stacking

In [33]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n.iloc[train_index] 
        y_tr = y_train_n.iloc[train_index] 
        X_te = X_train_n.iloc[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

## Inference

In [34]:
# CV스태킹 알고리즘 각 모델에 적용
knn_train, knn_test = get_stacking_base_datasets(knn_clf, x_train, y_train, test_x, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, x_train, y_train, test_x, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, x_train, y_train, test_x,  7)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, x_train, y_train, test_x, 7)

# CV스태킹 알고리즘 결과로 메타 모델 학습/시험에 필요한 result_a result_b 만들기 
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)

KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


In [35]:
lr_final = LogisticRegression(C=10)
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

## Submit

In [36]:
submit = pd.read_csv('./sample_submission.csv')

In [37]:
submit['Y_Class'] = stack_final

In [38]:
submit.to_csv('./KFold2_submission.csv', index=False)